In [1]:
import numpy as np
from math import inf

In [2]:
GRID_SIZE = 4
PIECES_NUMBER = 16
EMPTY_POSITION = -1

In [3]:
from enum import Enum, auto

class Player(Enum):
    MAX = auto()
    MIN = auto()
    
    def toggle(self):
        if self == Player.MAX:
            return Player.MIN
        else:
            return Player.MAX
        
import copy

class Piece:
    """Definition of a game piece by 4 characteristics:
    - RoundShape [True/False]
    - BigSize [True/False]
    - LightColor [True/False]
    - TopHole [True/False]"""

    def __init__(self,
                 id = 1,
                 round_shape = False,
                 big_size = False,
                 light_color = False,
                 top_hole = False):
        self.round_shape = round_shape
        self.big_size = big_size
        self.light_color = light_color
        self.top_hole = top_hole
        self.id = id
        
    @staticmethod
    def get_piece_by_id(id):
        if id != 0:
            return pieces_dict[id]
    
    def __hash__(self):
        return (4 * (self.round_shape + 1) + 3 * (self.big_size + 1) + 2 * self.light_color + self.top_hole)

    @staticmethod 
    def to_string(piece_id):

        if piece_id == 0:
            return "No piece to place"

        if piece_id == -1 :
            return " . "

        piece_display = str(piece_id)
        piece = Piece.get_piece_by_id(piece_id)
        if piece.round_shape:
            if piece.top_hole:
                piece_display = "○"
            else:
                piece_display = "●"
        else:
            if piece.top_hole:
                piece_display = "□"
            else:
                piece_display = "■"

        if piece.big_size:
            piece_display = "B" + piece_display
        else:
            piece_display = "S" + piece_display

        if piece.light_color:
            piece_display = "W" + piece_display
        else:
            piece_display = "B" + piece_display

        return piece_display
    
    def __repr__(self):
        return Piece.to_string(self.id)
            

pieces_list_definition = [
        Piece(1, False, False, False, False),
        Piece(2, True, False, False, False),
        Piece(3, False, True, False, False),
        Piece(4, True, True, False, False),
        Piece(5, False, False, True, False),
        Piece(6, True, False, True, False),
        Piece(7, False, True, True, False),
        Piece(8, True, True, True, False),
        Piece(9, False, False, False, True),
        Piece(10, True, False, False, True),
        Piece(11, False, True, False, True),
        Piece(12, True, True, False, True),
        Piece(13, False, False, True, True),
        Piece(14, True, False, True, True),
        Piece(15, False, True, True, True),
        Piece(16, True, True, True, True)
    ]

pieces_dict = {
    1 : Piece(1, False, False, False, False),
    2 : Piece(2, True, False, False, False),
    3 : Piece(3, False, True, False, False),
    4 : Piece(4, True, True, False, False),
    5 : Piece(5, False, False, True, False),
    6 : Piece(6, True, False, True, False),
    7 : Piece(7, False, True, True, False),
    8 : Piece(8, True, True, True, False),
    9 : Piece(9, False, False, False, True),
    10: Piece(10, True, False, False, True),
    11: Piece(11, False, True, False, True),
    12: Piece(12, True, True, False, True),
    13: Piece(13, False, False, True, True),
    14: Piece(14, True, False, True, True),
    15: Piece(15, False, True, True, True),
    16: Piece(16, True, True, True, True)
}

str_to_pieceid = {
 'BS■': 1,
 'BS●': 2,
 'BB■': 3,
 'BB●': 4,
 'WS■': 5,
 'WS●': 6,
 'WB■': 7,
 'WB●': 8,
 'BS□': 9,
 'BS○': 10,
 'BB□': 11,
 'BB○': 12,
 'WS□': 13,
 'WS○': 14,
 'WB□': 15,
 'WB○': 16
 }



In [4]:
class Action:
    pass

# Instead of dealing with messy initializers for Turn, that is more reasonable to create two stages of a turn:
# a stage where we place a piece, if there is one; and a stage where we choose a piece.
class TurnStage:
    def __init__(self, piece, coord):
        self.piece = piece
        self.coord = coord

    def __hash__(self):
        return hash(self.piece + 1) + hash(self.coord)

    def __eq__(self, other):
        return type(self) == type(other) and self.piece == other.piece and self.coord == other.coord

class Turn(Action):
    """
    One turn is to put a piece on the board and pick a new one
    """

    def __init__(self, player = Player.MAX, placingStage = None, choosingStage = None):
        self.player = player
        if placingStage:
            self.placingStage = placingStage
        else:
            self.placingStage = TurnStage(0, None)

        if choosingStage:
            self.choosingStage = choosingStage
        else:
            self.choosingStage = TurnStage(0, None)
        
    def __hash__(self):
        return hash(self.player) + hash(self.placingStage) + hash(self.choosingStage)

    def __eq__(self, other):
        return (self.player == other.player and self.placingStage == other.placingStage and self.choosingStage == other.choosingStage)

    def __repr__(self):
        return f"Place {Piece.to_string(self.placingStage.piece)} to {self.placingStage.coord} and choose {Piece.to_string(self.choosingStage.piece)}"

In [5]:
class TerminalTest:
    def __init__(self):
        self._utilities = {}

    def utility(self, state):
        return self._utilities[state]

    def is_terminal(self, state):
        pass

class BoardTerminalTest(TerminalTest):
    def is_terminal(self, board):
        
        player = board.game_turn.player
        
        if self.check_rows_winning(board) or self.check_columns_winning(board) or self.check_diags_winning(board):
            self._utilities[board] = 1 if player == Player.MAX else -1
            return True

        if self.check_draw(board):
            self._utilities[board] = 0
            return True
        
        return False
        
    def _check_line_winning(self, line):
        if EMPTY_POSITION not in line:
            
            pieces = np.array([Piece.get_piece_by_id(p) for p in line]) #[piece_1, piece_2, piece_3, piece_4]
            prop = np.array([0, 0, 0, 0]) #round, big, light, hole
            
            for piece in pieces: 
                if piece.round_shape:
                    prop[0] += 1
                if piece.big_size:
                    prop[1] += 1
                if piece.light_color:
                    prop[2] += 1
                if piece.top_hole:
                    prop[3] += 1

            # either they all share the same property
            # or they dont share a single one 
            if 4 in prop or 0 in prop: #max(prop) == GRID_SIZE or max(prop) == 0: 
                return True
            else: 
                return False

        return False
    
    def check_draw(self, board):
        return not (board.grid == EMPTY_POSITION).any()

    def check_rows_winning(self, board):
        grid = board.grid
        for row in range(GRID_SIZE):
            if self._check_line_winning(grid[row]):
                return True

        return False

    def check_columns_winning(self, board):
        grid = board.grid
        for col in range(GRID_SIZE):
            if self._check_line_winning(grid[:,col]):
                return True

        return False

    def check_diags_winning(self, board):
        grid = board.grid
        if self._check_line_winning(grid.diagonal()) \
                or self._check_line_winning(np.fliplr(grid).diagonal()):
            return True

        return False

    

In [6]:
class State:
    def get_player(self):
        pass

    def get_applicable_actions(self):
        pass

    def get_action_result(self, action):
        pass
    
    # Method that adds equatability
    def __eq__(self, other):
        pass
    
    # Method that makes the object hashable
    def __hash__(self):
        pass

class Board(State):
    """Definition of all data of the game at an instant:
    - grid [2 dimension list]
    - game_turn [GameTurn]"""

    def __init__(self, grid = None, game_turn = None):
        self.boardTerminalTest = BoardTerminalTest()
        if grid is not None:
            self.grid = grid
        else:
            self.grid = self.init_grid()
            
        if game_turn:
            self.game_turn = game_turn
        else:
            self.game_turn = Turn() ## choosing randomly for a player
        
        
    ## Implementation of State methods
    def get_player(self):
        return self.game_turn.player
    
    def get_applicable_actions(self):
        actions = set()
        
        piece_to_place = self.game_turn.placingStage.piece
        current_player = self.get_player()
        next_player = current_player.toggle()
        all_pieces = set(range(1, 17))
        remaining_pieces = all_pieces - set(np.unique(self.grid)) - set([piece_to_place])

        # optimize with numpy as np.where
        if piece_to_place == 0:
            for remaining_piece in remaining_pieces:

                placingStage = None

                choosingStage = TurnStage(remaining_piece, None)

                turn = Turn(player = next_player, placingStage = placingStage, choosingStage = choosingStage)

                actions.add(turn)

        elif len(remaining_pieces) != 0:
            coords = np.where(self.grid == EMPTY_POSITION)
            for i in range(len(coords[0])):
                for remaining_piece in remaining_pieces:
                    piece_to_place_coord = (coords[1][i], coords[0][i])

                    placingStage = TurnStage(piece_to_place, piece_to_place_coord)

                    choosingStage = TurnStage(remaining_piece, None)

                    turn = Turn(player = next_player, placingStage = placingStage, choosingStage = choosingStage)

                    actions.add(turn)

        else:
            empty_pos_coord = np.where(self.grid == EMPTY_POSITION)

            row, col = empty_pos_coord[0][0], empty_pos_coord[1][0]

            piece_to_place_coord = (col, row)

            placingStage = TurnStage(piece_to_place, piece_to_place_coord)

            choosingStage = None

            turn = Turn(player = next_player, placingStage = placingStage, choosingStage = choosingStage)
            actions.add(turn)

        return actions

    def get_action_result(self, turn):

        new_grid = copy.deepcopy(self.grid)

        next_game_turn = Turn(player = turn.player, placingStage = turn.choosingStage, choosingStage = None)

        if turn.placingStage:
            piece_to_place = turn.placingStage.piece
            piece_to_place_coord = turn.placingStage.coord
            
            if piece_to_place_coord: ## not during the first iteration, when a player just gives a piece, without putting one
                col, row = piece_to_place_coord[0], piece_to_place_coord[1]
                new_grid[row, col] = piece_to_place
        
        new_board = Board(grid = new_grid, game_turn = next_game_turn)
        
        return new_board
        
    # Method that adds equatability
    def __eq__(self, other):
        return type(self) == type(other) and self.game_turn == other.game_turn and self.check_for_symmetry(withOther = other.grid)
    
    # Method that makes the object hashable
    # TODO: make hash faster 
    def __hash__(self):
        return sum([hash(j) for i in self.grid for j in i]) * hash(self.game_turn)

    def init_grid(self):
        return np.zeros((4,4)) + EMPTY_POSITION

    def check_position_availability(self, x, y):
        return self.grid[y, x] == EMPTY_POSITION

    def check_for_symmetry(self, withOther):
        for k in range(0, 4):
            rotated = np.rot90(self.grid, k = k)
            all_same_transposed = (withOther == rotated.T).all()
            all_same_rotated = (withOther == rotated).all()
            
            if all_same_transposed or all_same_rotated:
                return True
        
        return False

    def has_selected_piece(self):
        return self.game_turn.selected_piece > 0

    def __repr__(self):
        legend_string = f"Piece to place {Piece.to_string(self.game_turn.placingStage.piece)}\n"  ##"Example, BS■ - Black Small Filled Square\n"
        display_string = '    A    B    C    D\n'
        for i, row in enumerate(self.grid, start = 1):
            display_string += ' '
            display_string += str(i)
            display_string += ' '
            for position in row:
                display_string += Piece.to_string(position)
                display_string += '  '
            display_string += '\n'
        return legend_string + display_string + '\n\n'



In [7]:
class Search:
    
    def __init__(self, optimized = False):
        # We will hold the number of nodes as a variable and return in the function below
        self._number_of_generated_states = 0
        self.state_utilities = {}
        self.optimized = optimized
        
    def max_value(self, state, terminal_test, strategy):
        self._number_of_generated_states = 0
        pass
    
    def min_value(self, state, terminal_test, strategy):
        self._number_of_generated_states = 0
        pass
        
    def find_strategy(self, initial_state, terminal_test):
        pass

    def set_utility_for(self, initial_state):
        player = initial_state.get_player()
        actions = initial_state.get_applicable_actions()

        states = {initial_state.get_action_result(action) for action in actions}
        utilities = [self.state_utilities[state] for state in states]
        if player == Player.MAX:
            self.state_utilities[initial_state] = max(utilities)
        else:
            self.state_utilities[initial_state] = min(utilities)
    
    # This function will simply return the instance member. 
    def number_of_generated_states(self):
        return self._number_of_generated_states    

class MinimaxSearch(Search):
    
    def find_strategy(self, initial_state, terminal_test):
        self.state_utilities = {}
        strategy = {}
        self._number_of_generated_states = 0
        self.max_value(initial_state, terminal_test, strategy)
        self.set_utility_for(initial_state)
        return strategy

    def max_value(self, state, terminal_test, strategy):
        # We check if it is the optimized version that we are calling. If yes, we keep (state, utility) pairs
        if self.optimized and state in self.state_utilities:
            return self.state_utilities[state]
        
        # Check for the terminal state. If it is, we store the utility and return it
        if terminal_test.is_terminal(state):
            utility = terminal_test.utility(state)
            strategy[state] = None
            self.state_utilities[state] = utility
            return utility
        
        # Explanation can be found from the pseudocode
        v = -inf
        move = None
        
        # Loop through available actions
        actions = state.get_applicable_actions()
        for action in actions:
            # Generate new state and increment the number of generated states
            new_state = state.get_action_result(action)
            self._number_of_generated_states += 1
            
            # Recursive call to min_value
            v2 = self.min_value(new_state, terminal_test, strategy)
            
            # If we found a utility greater than the current one, we update it as well as the action
            if v2 > v:
                v = v2
                move = action
        
            # At the end we store the pairs (state, action) and (state, utility)
            strategy[state] = move
            self.state_utilities[new_state] = v
            
        return v
    
    # Implementation and the explanation is nearly identical to the one above.
    def min_value(self, state, terminal_test, strategy):
        if self.optimized and state in self.state_utilities:
            return self.state_utilities[state]
        
        if terminal_test.is_terminal(state):
            utility = terminal_test.utility(state)
            strategy[state] = None
            self.state_utilities[state] = utility
            return utility
            
        v = inf
        move = None
        
        actions = state.get_applicable_actions()

        for action in actions:
            new_state = state.get_action_result(action)
            self._number_of_generated_states += 1
            v2 = self.max_value(new_state, terminal_test, strategy)
            if v2 < v:
                v = v2
                move = action
        
            strategy[state] = move
            self.state_utilities[new_state] = v  
        
        return v

class AlphaBetaSearch(Search):
    
    def find_strategy(self, initial_state, terminal_test):
        strategy = {}
        self.state_utilities = {}
        self._number_of_generated_states = 0
        self.max_value(initial_state, terminal_test, -inf, inf, strategy)
        self.set_utility_for(initial_state)
        return strategy

    def max_value(self, state, terminal_test, alpha, beta, strategy):
        
        if self.optimized and state in self.state_utilities:
            return self.state_utilities[state]
        
        if terminal_test.is_terminal(state):
            utility = terminal_test.utility(state)
            strategy[state] = None
            self.state_utilities[state] = utility
            return utility
        
        # The advantages of alpha-beta start to arise here. We first make a symbolical reassignment of alpha
        alpha_1 = alpha
        v = -inf
        move = None
        
        actions = state.get_applicable_actions()

        assert len(actions) != 0, f"No actions were found and terminal test did not fire {state}" 

        for action in actions:
            new_state = state.get_action_result(action)
            self._number_of_generated_states += 1
            v2 = self.min_value(new_state, terminal_test, alpha_1, beta, strategy)
            
            if v2 > v:
                v = v2
                move = action
                # Here, we update the value of alpha if we found a greater one
                alpha_1 = max(alpha_1, v)
                
            # And here is the power of alpha-beta. If we found that our current utility is greater than beta,
            # We will simply return the utility, thus pruning the other states.
            if v >= beta:
                strategy[state] = move
                self.state_utilities[new_state] = v
                return v

            strategy[state] = move
            self.state_utilities[new_state] = v
        
        return v
    
    # Analogically to the method above, we implement min_value method
    def min_value(self, state, terminal_test, alpha, beta, strategy):
        
        if self.optimized and state in self.state_utilities:
            return self.state_utilities[state]
        
        if terminal_test.is_terminal(state):
            utility = terminal_test.utility(state)
            strategy[state] = None
            self.state_utilities[state] = utility
            return utility
        
        v = inf
        beta_1 = beta
        move = None
        
        actions = state.get_applicable_actions()

        assert len(actions) != 0, "No actions were found and terminal test did not fire"

        for action in actions:
            new_state = state.get_action_result(action)
            self._number_of_generated_states += 1
            v2 = self.max_value(new_state, terminal_test, alpha, beta_1, strategy)
            
            if v2 < v:
                v = v2
                move = action
                beta_1 = min(beta_1, v)

            if v <= alpha:
                strategy[state] = move
                self.state_utilities[new_state] = v  
        
                return v

            strategy[state] = move
            self.state_utilities[new_state] = v  
        
        return v

In [8]:
def play(board):
    board_ = board
    
    try:
        best_action = strategy[board_]
        
    except KeyError:
        
        print("The end")
        best_action = None
        
    if best_action:
        print("Best action in this case is")
        print(best_action)
        
        board_ = board.get_action_result(best_action)
        actions = list(board_.get_applicable_actions())
        
        for i in range(len(actions)):
            print(f"{i} {actions[i]}", sep = "\n")
            
        print()
        
        if not terminalTest.is_terminal(board) and len(actions) != 0:
            selected_action_id = int(input("What did the opponent do? "))
            print()
            try:
                board_ = board_.get_action_result(actions[selected_action_id])
                play(board_)
            except IndexError:
                print("The end")
                

# WS■ - White, Small, Square, No hole
# BS○  - Black, Small, Circle, Holed

In [9]:
str_to_pieceid

{'BS■': 1,
 'BS●': 2,
 'BB■': 3,
 'BB●': 4,
 'WS■': 5,
 'WS●': 6,
 'WB■': 7,
 'WB●': 8,
 'BS□': 9,
 'BS○': 10,
 'BB□': 11,
 'BB○': 12,
 'WS□': 13,
 'WS○': 14,
 'WB□': 15,
 'WB○': 16}

In [20]:
grid = [['XXX', 'XXX', 'XXX', 'XXX',],
        ['XXX', 'XXX', 'XXX', 'XXX',],
        ['XXX', 'XXX', 'XXX', 'XXX',],
        ['XXX', 'XXX', 'XXX', 'XXX',]]
piece_to_place = "XXX"


for row in range(4):
    for col in range(4):
        elem = grid[row][col]
        if elem != "XXX":
            grid[row][col] = str_to_pieceid[elem]
        else:
            grid[row][col] = EMPTY_POSITION
grid = np.array(grid)
piece_to_place = str_to_pieceid[piece_to_place]

game_turn = Turn(player = Player.MAX, placingStage = TurnStage(piece_to_place, None))
board = Board(grid = grid, game_turn = game_turn)
terminalTest = BoardTerminalTest()
board

Piece to place BS●
    A    B    C    D
 1 WB○  BB□  BS□  BB■  
 2  .    .   WB●  WS●  
 3 BS○  WS□  WS○  WB■  
 4  .    .    .   BB○  



In [21]:
search = AlphaBetaSearch(optimized = True)
terminalTest = BoardTerminalTest()
strategy = search.find_strategy(board, terminalTest)

In [24]:
play(board)

Best action in this case is
Place BS● to (1, 1) and choose BS■
0 Place BS■ to (2, 3) and choose BB●
1 Place BS■ to (2, 3) and choose WS■
2 Place BS■ to (0, 3) and choose WB□
3 Place BS■ to (1, 3) and choose WB□
4 Place BS■ to (2, 3) and choose WB□
5 Place BS■ to (1, 3) and choose BB●
6 Place BS■ to (1, 3) and choose WS■
7 Place BS■ to (0, 1) and choose BB●
8 Place BS■ to (0, 1) and choose WS■
9 Place BS■ to (0, 3) and choose BB●
10 Place BS■ to (0, 3) and choose WS■
11 Place BS■ to (0, 1) and choose WB□



KeyboardInterrupt: Interrupted by user